In [19]:
import numpy as np
import csv
import time

np.random.seed(1234)

def randomize():
        np.random.seed(time.time())

In [20]:
# define hyperparameter

RND_MEAN = 0
RND_STD = 0.003
# RND_MEAN, RND_STE WILL BE USED FOR RANDOM DISTRIBUTION'S MEAN AND STD

LEARNING_RATE = 0.001

## SLP REGRESSION function call relation

<img src = "KakaoTalk_20220426_231100067.jpg" width = "700px">

In [21]:
# main function

def slp_regression(epoch_count = 10, mb_size = 10, report = 1, 
                   dataset_path = "C:/Users/user/Desktop/PythonLecture/Deep Learning Study/SLP/Regression_Analysis/archive/abalone.csv"):
    load_dataset(dataset_path)
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [22]:
# data load function

def load_dataset(dataset_path):
    with open(dataset_path) as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        
        for row in csvreader:
            rows.append(row)
    
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1
    
    data = np.zeros([len(rows), input_cnt + output_cnt])
    
    for n, row in enumerate(rows):
        if row[0] == 'I':
            data[n, 0] = 1
        if row[0] == 'M':
            data[n, 1] = 1
        if row[0] == 'W':
            data[n, 2] = 1
        
        data[n, 3:] = row[1:]


In [31]:
# initialize parameter function

def init_model():
    global weight, bias, input_cnt, output_cnt
    
    # randomly initialize weights
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    
    bias = np.zeros([output_cnt])

In [24]:
# train and test function

def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch + 1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {} : loss = {:5.3f}, accuracy = {:5.3f}/{:5.3f}' .format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\n Final Test : final accuracy = {:5.3f}' .format(final_acc))

In [25]:
# arrange data function
# randomly shuffle data

def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    
    # shuffle_map : Serial Number
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    
    # data.shape[0] * 0.8 : number of train data
    # step_count : iteration number for training
    step_count = int(data.shape[0] * 0.8) // mb_size
    
    test_begin_idx = step_count * mb_size
    
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    
    test_data = data[shuffle_map[test_begin_idx:]]
    
    # return test data set by using shuffle_map(randomly shuffled serial number -> extract random datas from dataset)
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth : mb_size*(nth + 1)]]
    
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [37]:
# run train function
# One step learning is performed using the mini batch input matrix and the correct answer matrix given as part of the training data.
def run_train(x, y):
    
    # perform forward propagation on the SLP neural network to obtain the neural network output
    output, aux_nn = forward_neuralnet(x)
    
    # loss function is calculated from the output and y through a post processing forward propagation task tailored to the nature of the regression analysis ploblem
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    # while the backprop_neuralnet() function is being executed, the neural network parameter value changes (Learning actually happens)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

# run test function
def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    
    return accuracy


In [35]:
# forward propagation
# forward neuralnet function
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    
    return output, x

# back propagation neuralnet function
def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis = 0)
    
    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b
    

In [33]:
def forward_postproc(output, y):
    diff = output - y
    
    # MSE
    square = np.square(diff)
    loss = np.mean(square)
    
    return loss, diff

def backprop_postproc(G_loss, diff):
    shape = diff.shape
    
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1
    
    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output    

In [29]:
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y)/y))
    
    return 1 - mdiff

In [38]:
slp_regression()

Epoch 1 : loss = 35.123, accuracy = 0.547/0.802
Epoch 2 : loss = 8.801, accuracy = 0.815/0.804
Epoch 3 : loss = 8.055, accuracy = 0.806/0.800
Epoch 4 : loss = 7.940, accuracy = 0.804/0.798
Epoch 5 : loss = 7.848, accuracy = 0.803/0.799
Epoch 6 : loss = 7.768, accuracy = 0.804/0.798
Epoch 7 : loss = 7.694, accuracy = 0.803/0.800
Epoch 8 : loss = 7.626, accuracy = 0.805/0.799
Epoch 9 : loss = 7.564, accuracy = 0.805/0.800
Epoch 10 : loss = 7.506, accuracy = 0.805/0.800

 Final Test : final accuracy = 0.800


In [39]:
print(weight)

[[ 5.69804204e-01]
 [ 8.06789894e-01]
 [-1.88280559e-04]
 [ 2.23920465e+00]
 [ 1.77663975e+00]
 [ 6.51870105e-01]
 [ 2.71027446e+00]
 [ 6.44969946e-01]
 [ 5.52622584e-01]
 [ 1.12557761e+00]]


In [40]:
print(bias)

[4.5177512]


In [41]:
LEARNING_RATE = 0.1
slp_regression()

Epoch 1 : loss = 7.705, accuracy = 0.802/0.833
Epoch 2 : loss = 6.490, accuracy = 0.815/0.833
Epoch 3 : loss = 6.068, accuracy = 0.821/0.852
Epoch 4 : loss = 5.829, accuracy = 0.824/0.854
Epoch 5 : loss = 5.704, accuracy = 0.826/0.855
Epoch 6 : loss = 5.564, accuracy = 0.829/0.857
Epoch 7 : loss = 5.470, accuracy = 0.829/0.820
Epoch 8 : loss = 5.449, accuracy = 0.831/0.820
Epoch 9 : loss = 5.481, accuracy = 0.830/0.832
Epoch 10 : loss = 5.424, accuracy = 0.830/0.854

 Final Test : final accuracy = 0.854
